<a href="https://colab.research.google.com/github/vinnyvalenz/Inspirational-Quote-Generator/blob/HRomero/inspirational_quote_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import pandas as pd


In [ ]:
from google.colab import files
import io
uploaded_item = files.upload()

Saving Motivational Quotes Database.csv to Motivational Quotes Database.csv


In [ ]:
 # Read in csv
df = pd.read_csv('Motivational Quotes Database.csv', encoding = "ISO-8859-1")
df.rename(columns = {'Motivational Quotes Database - https://www.sharpquotes.com':'Quotes'}, inplace = True)
#quotes = df.pop("Quotes")


df = df.tail(20000)
#quotes = quotes.str.lower()
filtered = df["Quotes"].to_string()

#tokenizer = RegexpTokenizer(r'\w+')
#tokens = tokenizer.tokenize(quotes["Quotes"])

#stop = stopwords.words('english')
#quotes.apply(lambda x: [item for item in x if item not in stop])
#quotes = quotes["Quotes"].to_string())
#filtered.replace('\n', ' ')

In [ ]:
filtered

'25578    I love and always have loved policy issues and...\n25579    My politics are of a practical kind - the inte...\n25580    Aggression is simply another name for government.\n25581    For me, journalism has been more a matter of p...\n25582    I have long said there are three distinct grou...\n25583    We are all used to paying a sales tax when we ...\n25584    Health care reform, the marquee legislative ac...\n25585    The reality of split government puts a premium...\n25586    You can\'t have a situation in which companies ...\n25587    As more people rely on government programs, th...\n25588    A wise government knows how to enforce with te...\n25589    I\'m not driven to get back into politics. It\'s...\n25590    I believe that government is the servant of th...\n25591    I love my government not least for the extent ...\n25592    Government is either organized benevolence or ...\n25593    The money that goes into Social Security is no...\n25594    A reporter\'s ability to ke

In [ ]:
#filtered = filtered.replace('\n\d+', ' ')#removes "\n"
#filtered = filtered.replace(r'^\d+\.\s+','') #removes numbers
#tokenizer = RegexpTokenizer('\w+')
#tokens = tokenizer.tokenize(filtered.lower())
#filtered = filter(lambda token: token not in stopwords.words('english'), tokens)

In [ ]:
#processing data
filtered = ''.join([i for i in filtered if not i.isdigit()])
filtered = filtered.lower()


# instantiate the tokenizer
tokenizer = RegexpTokenizer(r'\w+')
filtered = tokenizer.tokenize(filtered)

# if the created token isn't in the stop words, make it part of "filtered"
#filtered = filter(lambda token: token not in stopwords.words('english'), tokens)

In [ ]:
filtered = " ".join(filtered)

In [ ]:
filtered

'i love and always have loved policy issues and my politics are of a practical kind the inte aggression is simply another name for government for me journalism has been more a matter of p i have long said there are three distinct grou we are all used to paying a sales tax when we health care reform the marquee legislative ac the reality of split government puts a premium you can t have a situation in which companies as more people rely on government programs th a wise government knows how to enforce with te i m not driven to get back into politics it s i believe that government is the servant of th i love my government not least for the extent government is either organized benevolence or the money that goes into social security is no a reporter s ability to keep the bond of confi urban america has been redlined government ha we ve been so preoccupied with getting the gov we oppose the reactionary policies of the u s is there any wonder why we are in such big tro and i just think that 

#All code below was taken from https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/
#slight modifications to better suit this project


In [ ]:
# create mapping
chars = sorted(list(set(filtered)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [ ]:
input_len = len(filtered)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 910782
Total vocab: 27


In [ ]:
# prepare the dataset
seq_length = 100
x_data = []
y_data = []

In [ ]:
for i in range(0, input_len - seq_length, 1):
  in_seq = filtered[i:i + seq_length]
  out_seq = filtered[i + seq_length]
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

In [ ]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 910682


In [ ]:
# reshape X and normalize
X = np.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [ ]:
# one hot encode
y = np_utils.to_categorical(y_data)

In [ ]:
# define LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
# load network weights, use "adam"
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
model.fit(X, y, epochs=10, batch_size=256, callbacks=desired_callbacks)

Epoch 1/10
3558/3558 [==============================] - 718s 201ms/step - loss: 2.5358

Epoch 00001: loss improved from inf to 2.23592, saving model to model_weights_saved.hdf5
Epoch 2/10
3558/3558 [==============================] - 721s 203ms/step - loss: 1.7722

Epoch 00002: loss improved from 2.23592 to 1.72733, saving model to model_weights_saved.hdf5
Epoch 3/10
3558/3558 [==============================] - 722s 203ms/step - loss: 1.6140

Epoch 00003: loss improved from 1.72733 to 1.59388, saving model to model_weights_saved.hdf5
Epoch 4/10
3558/3558 [==============================] - 721s 203ms/step - loss: 1.5284

Epoch 00004: loss improved from 1.59388 to 1.51869, saving model to model_weights_saved.hdf5
Epoch 5/10
3558/3558 [==============================] - 721s 203ms/step - loss: 1.4794

Epoch 00005: loss improved from 1.51869 to 1.47115, saving model to model_weights_saved.hdf5
Epoch 6/10
3558/3558 [==============================] - 722s 203ms/step - loss: 1.4363

Epoch 00006

In [ ]:
#loads weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
#uses random seed
start = np.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" and get the best medi i have many times thought i did the wrong thin canadians have been very genero "


In [ ]:
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = num_to_char[index]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

us the most people who have a great man is the most people who have a great man is the most people w